# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 6 2022 10:55AM,241199,15,VT80174788,"Virtus Pharmaceuticals, LLC",Released
1,Jul 6 2022 10:54AM,241195,15,8234685,"Mizner Bioscience, LLC",Released
2,Jul 6 2022 10:53AM,241197,15,8235273,"Carwin Pharmaceutical Associates, LLC",Released
3,Jul 6 2022 10:53AM,241197,15,8235274,"Carwin Pharmaceutical Associates, LLC",Released
4,Jul 6 2022 10:53AM,241197,15,8238522,"Carwin Pharmaceutical Associates, LLC",Released
5,Jul 6 2022 10:53AM,241197,15,8238523,"Carwin Pharmaceutical Associates, LLC",Released
6,Jul 6 2022 10:53AM,241197,15,8238524,"Carwin Pharmaceutical Associates, LLC",Released
7,Jul 6 2022 10:52AM,241196,15,8235268,"Laser Pharmaceuticals, LLC",Released
8,Jul 6 2022 10:52AM,241196,15,8235872,"Laser Pharmaceuticals, LLC",Released
9,Jul 6 2022 10:52AM,241196,15,8235873,"Laser Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
66,241195,Released,1
67,241196,Released,4
68,241197,Released,5
69,241198,Released,1
70,241199,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241195,NaN,NaN,1.0
241196,NaN,NaN,4.0
241197,NaN,NaN,5.0
241198,NaN,NaN,1.0
241199,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241087,0.0,15.0,15.0
241095,0.0,0.0,8.0
241100,0.0,0.0,1.0
241101,0.0,2.0,2.0
241103,0.0,3.0,12.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241087,0,15,15
241095,0,0,8
241100,0,0,1
241101,0,2,2
241103,0,3,12


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241087,0,15,15
1,241095,0,0,8
2,241100,0,0,1
3,241101,0,2,2
4,241103,0,3,12


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241087,,15,15
1,241095,,,8
2,241100,,,1
3,241101,,2,2
4,241103,,3,12


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 6 2022 10:55AM,241199,15,"Virtus Pharmaceuticals, LLC"
1,Jul 6 2022 10:54AM,241195,15,"Mizner Bioscience, LLC"
2,Jul 6 2022 10:53AM,241197,15,"Carwin Pharmaceutical Associates, LLC"
7,Jul 6 2022 10:52AM,241196,15,"Laser Pharmaceuticals, LLC"
11,Jul 6 2022 10:52AM,241198,19,"Avet Pharmaceuticals Labs, Inc"
12,Jul 6 2022 10:42AM,241193,15,"Brookfield Pharmaceuticals, LLC"
32,Jul 6 2022 10:40AM,241192,10,Eywa Pharma Inc.
49,Jul 6 2022 10:38AM,241190,15,"Alliance Pharma, Inc."
86,Jul 6 2022 10:30AM,241191,10,ISDIN Corporation
88,Jul 6 2022 10:30AM,241189,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 6 2022 10:55AM,241199,15,"Virtus Pharmaceuticals, LLC",,,1
1,Jul 6 2022 10:54AM,241195,15,"Mizner Bioscience, LLC",,,1
2,Jul 6 2022 10:53AM,241197,15,"Carwin Pharmaceutical Associates, LLC",,,5
3,Jul 6 2022 10:52AM,241196,15,"Laser Pharmaceuticals, LLC",,,4
4,Jul 6 2022 10:52AM,241198,19,"Avet Pharmaceuticals Labs, Inc",,,1
5,Jul 6 2022 10:42AM,241193,15,"Brookfield Pharmaceuticals, LLC",,5,15
6,Jul 6 2022 10:40AM,241192,10,Eywa Pharma Inc.,,,17
7,Jul 6 2022 10:38AM,241190,15,"Alliance Pharma, Inc.",,,37
8,Jul 6 2022 10:30AM,241191,10,ISDIN Corporation,,,2
9,Jul 6 2022 10:30AM,241189,10,ISDIN Corporation,,,46


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 10:55AM,241199,15,"Virtus Pharmaceuticals, LLC",1,,
1,Jul 6 2022 10:54AM,241195,15,"Mizner Bioscience, LLC",1,,
2,Jul 6 2022 10:53AM,241197,15,"Carwin Pharmaceutical Associates, LLC",5,,
3,Jul 6 2022 10:52AM,241196,15,"Laser Pharmaceuticals, LLC",4,,
4,Jul 6 2022 10:52AM,241198,19,"Avet Pharmaceuticals Labs, Inc",1,,
5,Jul 6 2022 10:42AM,241193,15,"Brookfield Pharmaceuticals, LLC",15,5,
6,Jul 6 2022 10:40AM,241192,10,Eywa Pharma Inc.,17,,
7,Jul 6 2022 10:38AM,241190,15,"Alliance Pharma, Inc.",37,,
8,Jul 6 2022 10:30AM,241191,10,ISDIN Corporation,2,,
9,Jul 6 2022 10:30AM,241189,10,ISDIN Corporation,46,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 10:55AM,241199,15,"Virtus Pharmaceuticals, LLC",1,,
1,Jul 6 2022 10:54AM,241195,15,"Mizner Bioscience, LLC",1,,
2,Jul 6 2022 10:53AM,241197,15,"Carwin Pharmaceutical Associates, LLC",5,,
3,Jul 6 2022 10:52AM,241196,15,"Laser Pharmaceuticals, LLC",4,,
4,Jul 6 2022 10:52AM,241198,19,"Avet Pharmaceuticals Labs, Inc",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 10:55AM,241199,15,"Virtus Pharmaceuticals, LLC",1.0,NaN,NaN
1,Jul 6 2022 10:54AM,241195,15,"Mizner Bioscience, LLC",1.0,NaN,NaN
2,Jul 6 2022 10:53AM,241197,15,"Carwin Pharmaceutical Associates, LLC",5.0,NaN,NaN
3,Jul 6 2022 10:52AM,241196,15,"Laser Pharmaceuticals, LLC",4.0,NaN,NaN
4,Jul 6 2022 10:52AM,241198,19,"Avet Pharmaceuticals Labs, Inc",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 10:55AM,241199,15,"Virtus Pharmaceuticals, LLC",1.0,0.0,0.0
1,Jul 6 2022 10:54AM,241195,15,"Mizner Bioscience, LLC",1.0,0.0,0.0
2,Jul 6 2022 10:53AM,241197,15,"Carwin Pharmaceutical Associates, LLC",5.0,0.0,0.0
3,Jul 6 2022 10:52AM,241196,15,"Laser Pharmaceuticals, LLC",4.0,0.0,0.0
4,Jul 6 2022 10:52AM,241198,19,"Avet Pharmaceuticals Labs, Inc",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,7475893,450.0,2.0,3.0
12,1205666,26.0,4.0,0.0
15,1447170,63.0,5.0,0.0
16,1687865,9.0,7.0,0.0
18,241165,1.0,0.0,0.0
19,964587,58.0,42.0,5.0
20,964477,95.0,25.0,0.0
21,241183,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,7475893,450.0,2.0,3.0
1,12,1205666,26.0,4.0,0.0
2,15,1447170,63.0,5.0,0.0
3,16,1687865,9.0,7.0,0.0
4,18,241165,1.0,0.0,0.0
5,19,964587,58.0,42.0,5.0
6,20,964477,95.0,25.0,0.0
7,21,241183,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,450.0,2.0,3.0
1,12,26.0,4.0,0.0
2,15,63.0,5.0,0.0
3,16,9.0,7.0,0.0
4,18,1.0,0.0,0.0
5,19,58.0,42.0,5.0
6,20,95.0,25.0,0.0
7,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,450.0
1,12,Released,26.0
2,15,Released,63.0
3,16,Released,9.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21
Status,,,,,,,,
Completed,3.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
Executing,2.0,4.0,5.0,7.0,0.0,42.0,25.0,0.0
Released,450.0,26.0,63.0,9.0,1.0,58.0,95.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21
0,Completed,3.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
1,Executing,2.0,4.0,5.0,7.0,0.0,42.0,25.0,0.0
2,Released,450.0,26.0,63.0,9.0,1.0,58.0,95.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21
0,Completed,3.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
1,Executing,2.0,4.0,5.0,7.0,0.0,42.0,25.0,0.0
2,Released,450.0,26.0,63.0,9.0,1.0,58.0,95.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()